In [1]:
%load_ext autoreload
%autoreload 2

import molvis as mv
import molpy as mp

h2o = mp.Atomistic()
h2o.def_atom(id=0, name="O", element="O", xyz=[0, 0, 0])
h2o.def_atom(id=0, name="H1", element="H", xyz=[0.9572, 0, 0])
h2o.def_atom(id=0, name="H2", element="H", xyz=[-0.239987, 0.92705, 0])
h2o.def_bond(0, 1)
h2o.def_bond(0, 2)

canvas = mv.Molvis()
canvas.draw_frame(h2o.to_frame())

Molvis()

In [2]:
%load_ext autoreload
%autoreload 2

from molpy.builder.reacter_lammps import ReacterBuilder, ReactionTemplate, ReactantWrapper
from pathlib import Path
import molpy as mp
import molvis as mv
import numpy as np

workdir = Path("data/case2")

"""
Example usage: Generate reaction templates for nylon-6,6 synthesis.

This example demonstrates the condensation reaction between
adipic acid and hexamethylenediamine to form nylon-6,6.

Based on LAMMPS examples: https://github.com/lammps/lammps/tree/develop/examples/PACKAGES/reaction/nylon%2C6-6_melt
"""
# Load adipic acid structure (HOOC-(CH2)4-COOH)
adipic_acid_frame = mp.io.read_pdb(workdir/"adipicacid.pdb")
adipic_acid_struct = mp.Atomistic.from_frame(adipic_acid_frame, name="adipic_acid")
# adipic_acid_struct.gen_topo_items(is_angle=True, is_dihedral=True)

# Load hexamethylenediamine structure (H2N-(CH2)6-NH2)  
hexamethylenediamine_frame = mp.io.read_pdb(workdir/"hexamethylenediamine.pdb")
hexamethylenediamine_struct = mp.Atomistic.from_frame(hexamethylenediamine_frame, name="hexamethylenediamine")
# hexamethylenediamine_struct.gen_topo_items(is_angle=True, is_dihedral=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
builder = mp.builder.AmberToolsBuilder(workdir=workdir, conda_env="AmberTools25")
typifier = mp.builder.AmberToolsTypifier(workdir=workdir, conda_env="AmberTools25")

adipic_acid_struct = typifier.typify(adipic_acid_struct, is_prepi=True, is_frcmod=True, is_tleap=True)
hexamethylenediamine_struct = typifier.typify(hexamethylenediamine_struct, is_prepi=True, is_frcmod=True, is_tleap=True)
system = mp.System()
system.add_struct(adipic_acid_struct)
system.add_struct(hexamethylenediamine_struct)
system = typifier.parameterize("nylon66", system)
mp.io.write_lammps_forcefield(workdir/"nylon66.ff", system.forcefield)

Cleaned up temporary file: data/case2/adipic_acid/run_local.sh
Cleaned up temporary file: data/case2/adipic_acid/run_local.sh
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
Loading Prep file: /workspaces/molpy/examples/data/case2/adipic_acid/adipic_acid.prepi
Loading parameters: /workspaces/molpy/examples/data/case2/adipic_acid/adipic_acid.frcmod
Reading f

In [4]:
adipic_acid_reactant = ReactantWrapper(adipic_acid_struct)
hexamethylenediamine_reactant = ReactantWrapper(hexamethylenediamine_struct)
aa_template = adipic_acid_reactant.extract_fragment(ReactionTemplate(
        init_atom=1,     # Carboxyl carbon
        edge_atom=7,      # OH atoms to be deleted
    ))

hd_template = hexamethylenediamine_reactant.extract_fragment(ReactionTemplate(
        init_atom=21,     # Amino nitrogen
        edge_atom=12,    # H atom to be deleted
))
mp.SpatialWrapper(hd_template).move([-10, 1, -2])

stp1_pre = ReactantWrapper(
    mp.Atomistic.concat("stp1_pre", [aa_template._wrapped, hd_template._wrapped])
)
stp1_post = stp1_pre(name="stp1_post").def_bond(0, 8)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}


In [ ]:
stp1_pre["angles"][0]

KeyError: 'id'

In [5]:
builder = ReacterBuilder(workdir=workdir, typifier=typifier)
stp2_pre = builder.build("stp1", stp1_pre, stp1_post, inits=[6, 8], edges=[2, 9])
canvas.draw_frame(stp2_pre.to_frame())

Frame(
  [atoms] name: shape=(44,)
  [atoms] mass: shape=(44,)
  [atoms] type: shape=(44,)
  [atoms] xyz: shape=(44, 3)
  [atoms] element: shape=(44,)
  [atoms] id: shape=(44,)
  [atoms] q: shape=(44,)
  [bonds] j: shape=(42,)
  [bonds] i: shape=(42,)
  [bonds] type: shape=(42,)
  [angles] j: shape=(74,)
  [angles] i: shape=(74,)
  [angles] type: shape=(74,)
  [angles] k: shape=(74,)
  [dihedrals] i: shape=(120,)
  [dihedrals] l: shape=(120,)
  [dihedrals] type: shape=(120,)
  [dihedrals] k: shape=(120,)
  [dihedrals] j: shape=(120,)
)


KeyError: 'id'

In [ ]:
"bonds" in stp1_pre._wrapped.data

True

In [ ]:
stp2_pre["name"] = "stp2_pre"
stp2_post = stp2_pre(name="stp2_post").def_bond(6, 16)
stp2_post = stp2_post.del_bond(8, 16).del_bond(0, 6)

In [ ]:
reacted = builder.build("stp2", stp2_pre, stp2_post, inits=[6, 8], edges=[2, 9])

- Pre-reaction template: data/case2/stp2_pre.template

- Post-reaction template: data/case2/stp2_post.template



In [ ]:
import molpy.pack as mpk
box_size = np.array([40.0, 40.0, 40.0])  # 40 x 40 x 40 Angstrom box
box_origin = np.array([0.0, 0.0, 0.0])
packing_session = mpk.Session(packer="packmol")
box_constraint = mpk.InsideBoxConstraint(box_size, box_origin)
packing_session.add_target(
    frame=adipic_acid_struct.to_frame(),
    number=10,
    constraint=box_constraint
)
packing_session.add_target(
    frame=hexamethylenediamine_struct.to_frame(),
    number=10,
    constraint=box_constraint
)
frame = packing_session.optimize()
frame.box = mp.Box(box_size)
mp.io.write_lammps_data(workdir/"small_nylon_melt.data", frame)

Cleaned up temporary file: /workspaces/molpy/examples/run_local.sh


KeyError: "No variable named 'type'. Variables on the dataset include ['__class__', 'j', 'k', 'i', 'id']"

In [ ]:
frame["bonds"]

<xarray.Dataset> Size: 60kB
Dimensions:    (bonds_id: 420)
Dimensions without coordinates: bonds_id
Data variables:
    __class__  (bonds_id) <U25 42kB 'molpy.core.atomistic.Bond' ... 'molpy.co...
    j          (bonds_id) int64 3kB 0 0 0 2 3 4 2 ... 432 430 430 430 433 433
    i          (bonds_id) int64 3kB 1 14 18 1 1 1 5 ... 427 433 434 435 436 437
    type       (bonds_id) <U5 8kB 'c-c3' 'c-o' 'c-oh' ... 'c3-h1' 'c3-h1'
    id         (bonds_id) int64 3kB 1 2 3 4 5 6 7 ... 415 416 417 418 419 420